In [1]:
import sys
sys.executable

'/Users/eranagmon/code/process-bigraph/venv/bin/python'

In [2]:
import numpy as np
from process_bigraph import Process, ProcessTypes, Composite
import matplotlib.pyplot as plt
import cobra
from cobra.io import load_model
from scipy.ndimage import convolve

core = ProcessTypes()


In [3]:
# create new types
def apply_non_negative(schema, current, update, core):
    new_value = current + update
    return max(0, new_value)

positive_float = {
    '_type': 'positive_float',
    '_inherit': 'float',
    '_apply': apply_non_negative
}
core.register('positive_float', positive_float)

bounds_type = {
    'lower': 'maybe[float]',
    'upper': 'maybe[float]'
}
core.register_process('bounds', bounds_type)

In [4]:
core.access('positive_float')

{'_type': 'positive_float',
 '_check': 'check_float',
 '_apply': 'apply_non_negative',
 '_serialize': 'to_string',
 '_description': '64-bit floating point precision number',
 '_default': '0.0',
 '_deserialize': 'deserialize_float',
 '_divide': 'divide_float',
 '_dataclass': 'dataclass_float',
 '_inherit': ['float']}

## Dynamic FBA Process

In [5]:
class DynamicFBA(Process):
    """
    Performs dynamic FBA.

    Parameters:
    - model: The metabolic model for the simulation.
    - kinetic_params: Kinetic parameters (Km and Vmax) for each substrate.
    - biomass_reaction: The identifier for the biomass reaction in the model.
    - substrate_update_reactions: A dictionary mapping substrates to their update reactions.
    - biomass_identifier: The identifier for biomass in the current state.

    TODO -- check units
    """

    config_schema = {
        'model_file': 'string',
        'kinetic_params': 'map[tuple[float,float]]',
        'biomass_reaction': {
            '_type': 'string',
            '_default': 'Biomass_Ecoli_core'
        },
        'substrate_update_reactions': 'map[string]',
        'biomass_identifier': 'string',
        'bounds': 'map[bounds]',
    }

    def __init__(self, config, core):
        super().__init__(config, core)

        if not 'xml' in self.config['model_file']:
            # use the textbook model if no model file is provided
            self.model = load_model(self.config['model_file'])
        else:
            self.model = cobra.io.read_sbml_model(self.config['model_file'])

        for reaction_id, bounds in self.config['bounds'].items():
            if bounds['lower'] is not None:
                self.model.reactions.get_by_id(reaction_id).lower_bound = bounds['lower']
            if bounds['upper'] is not None:
                self.model.reactions.get_by_id(reaction_id).upper_bound = bounds['upper']

    def inputs(self):
        return {
            'substrates': 'map[positive_float]'
        }

    def outputs(self):
        return {
            'substrates': 'map[positive_float]'
        }

    # TODO -- can we just put the inputs/outputs directly in the function?
    def update(self, state, interval):
        substrates_input = state['substrates']

        for substrate, reaction_id in self.config['substrate_update_reactions'].items():
            Km, Vmax = self.config['kinetic_params'][substrate]
            substrate_concentration = substrates_input[substrate]
            uptake_rate = Vmax * substrate_concentration / (Km + substrate_concentration)
            self.model.reactions.get_by_id(reaction_id).lower_bound = -uptake_rate

        substrate_update = {}

        solution = self.model.optimize()
        if solution.status == 'optimal':
            current_biomass = substrates_input[self.config['biomass_identifier']]
            biomass_growth_rate = solution.fluxes[self.config['biomass_reaction']]
            substrate_update[self.config['biomass_identifier']] = biomass_growth_rate * current_biomass * interval

            for substrate, reaction_id in self.config['substrate_update_reactions'].items():
                flux = solution.fluxes[reaction_id]
                substrate_update[substrate] = flux * current_biomass * interval
                # TODO -- assert not negative?
        else:
            # Handle non-optimal solutions if necessary
            # print('Non-optimal solution, skipping update')
            for substrate, reaction_id in self.config['substrate_update_reactions'].items():
                substrate_update[substrate] = 0

        return {
            'substrates': substrate_update,
        }

core.register_process('DynamicFBA', DynamicFBA)

In [6]:
from process_bigraph.experiments.parameter_scan import RunProcess

def dfba_config(
        model_file='textbook',
        kinetic_params={
            'glucose': (0.5, 1),
            'acetate': (0.5, 2)},
        biomass_reaction='Biomass_Ecoli_core',
        substrate_update_reactions={
            'glucose': 'EX_glc__D_e',
            'acetate': 'EX_ac_e'},
        biomass_identifier='biomass',
        bounds={
            'EX_o2_e': {'lower': -2, 'upper': None},
            'ATPM': {'lower': 1, 'upper': 1}}
):
    return {
        'model_file': model_file,
        'kinetic_params': kinetic_params,
        'biomass_reaction': biomass_reaction,
        'substrate_update_reactions': substrate_update_reactions,
        'biomass_identifier': biomass_identifier,
        'bounds': bounds
    }


# TODO -- this should be imported, or just part of Process?
def run_process(
        address,
        config,
        core_type,
        initial_state,
        observables,
        timestep=1,
        runtime=10
):
    config = {
        'process_address': address,
        'process_config': config,
        'observables': observables,
        'timestep': timestep,
        'runtime': runtime}

    run = RunProcess(config, core_type)
    return run.update(initial_state)

In [7]:
n_bins = (5, 5)

initial_glucose = np.random.uniform(low=0, high=20, size=n_bins)
initial_acetate = np.random.uniform(low=0, high=0, size=n_bins)
initial_biomass = np.random.uniform(low=0, high=0.1, size=n_bins)

dfba_processes_dict = {}
for i in range(n_bins[0]):
    for j in range(n_bins[1]):
        dfba_processes_dict[f'[{i},{j}]'] = {
            '_type': 'process',
            'address': 'local:DynamicFBA',
            'config': dfba_config(),
            'inputs': {
                'substrates': {
                    'glucose': ['..', 'fields', 'glucose', i, j],
                    'acetate': ['..', 'fields', 'acetate', i, j],
                    'biomass': ['..', 'fields', 'biomass', i, j],
                }
            },
            'outputs': {
                'substrates': {
                    'glucose': ['..', 'fields', 'glucose', i, j],
                    'acetate': ['..', 'fields', 'acetate', i, j],
                    'biomass': ['..', 'fields', 'biomass', i, j]
                }
            }
        }

composite_state = {
    'fields': {
        '_type': 'map',
        '_value': {
            '_type': 'array',
            '_shape': n_bins,
            '_data': 'positive_float'
        },
        'glucose': initial_glucose,
        'acetate': initial_acetate,
        'biomass': initial_biomass,
    },
    'spatial_dfba': dfba_processes_dict,
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'fields': 'map',
                'time': 'float',
            }
        },
        'inputs': {
            'fields': ['fields'],
            'time': ['global_time']
        }
    }
}

sim = Composite({'state': composite_state}, core=core)

sim.update({}, 10.0)



[]

In [8]:
results = sim.gather_results()[('emitter',)]
results

[{'fields': {'glucose': array([[ 8.5526694 , 17.5842017 , 15.39583692,  7.60193967, 19.13203451],
          [12.65809497, 12.21049038,  5.7792075 , 10.33968228, 19.25409282],
          [ 3.14530227, 13.69455267,  1.14230224,  5.90495666, 10.12746262],
          [13.03804887, 19.10655932, 10.22930332,  3.11076213,  8.7102663 ],
          [ 1.26206527,  7.72923753,  3.72448763,  7.74081364,  3.94338213]]),
   'acetate': array([[0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0.]]),
   'biomass': array([[0.03124297, 0.0828025 , 0.06932402, 0.05347029, 0.06634931],
          [0.06510462, 0.01836797, 0.07575297, 0.09252517, 0.05390586],
          [0.03899395, 0.05617419, 0.00645221, 0.09252719, 0.02600646],
          [0.08406056, 0.08744284, 0.04027866, 0.01781626, 0.08739853],
          [0.08358264, 0.04685661, 0.01132015, 0.03643865, 0.02227665]])},
  'time': 0.0},
 {'fields': {'glucose': array

In [9]:
# plot results
def plot_results(results):
    fig, ax = plt.subplots()
    for key, value in results['substrates'].items():
        ax.plot(results['time'], value, label=key)
    ax.legend()
    plt.show()

In [10]:
# plot_results(results['results'])

## Diffusion Advection Process

In [11]:

# Laplacian for 2D diffusion
LAPLACIAN_2D = np.array([[0, 1, 0],
                         [1, -4, 1],
                         [0, 1, 0]])


class DiffusionAdvection(Process):
    config_schema = {
        'n_bins': 'tuple[integer,integer]',
        'bounds': 'tuple[float,float]',
        'default_diffusion_rate': {'_type': 'float', '_default': 1e-1},
        'default_diffusion_dt': {'_type': 'float', '_default': 1e-1},
        'diffusion_coeffs': 'map[float]',
        'advection_coeffs': 'map[tuple[float,float]]',
    }

    def __init__(self, config, core):
        super().__init__(config, core)

        # get diffusion rates
        bins_x = self.config['n_bins'][0]
        bins_y = self.config['n_bins'][1]
        length_x = self.config['bounds'][0]
        length_y = self.config['bounds'][1]
        dx = length_x / bins_x
        dy = length_y / bins_y
        dx2 = dx * dy

        # general diffusion rate
        diffusion_rate = self.config['default_diffusion_rate']
        self.diffusion_rate = diffusion_rate / dx2

        # diffusion rates for each individual molecules
        self.molecule_specific_diffusion = {
            mol_id: diff_rate / dx2
            for mol_id, diff_rate in self.config['diffusion_coeffs'].items()}

        # get diffusion timestep
        diffusion_dt = 0.5 * dx ** 2 * dy ** 2 / (2 * diffusion_rate * (dx ** 2 + dy ** 2))
        self.diffusion_dt = min(diffusion_dt, self.config['default_diffusion_dt'])

    def inputs(self):
        return {
            'fields': {
                '_type': 'map',
                '_value': {
                    '_type': 'array',
                    '_shape': self.config['n_bins'],
                    '_data': 'positive_float'
                },
            }
        }

    def outputs(self):
        return {
            'fields': {
                '_type': 'map',
                '_value': {
                    '_type': 'array',
                    '_shape': self.config['n_bins'],
                    '_data': 'positive_float'
                },
            }
        }

    def update(self, state, interval):
        fields = state['fields']

        fields_update = {}
        for species, field in fields.items():
            fields_update[species] = self.diffusion_delta(
                field,
                interval,
                diffusion_coeff=self.config['diffusion_coeffs'][species],
                advection_coeff=self.config['advection_coeffs'][species]
            )

        return {
            'fields': fields_update
        }

    def diffusion_delta(self, state, interval, diffusion_coeff, advection_coeff):
        t = 0.0
        dt = min(interval, self.diffusion_dt)
        updated_state = state.copy()

        while t < interval:

            # Diffusion
            laplacian = convolve(
                updated_state,
                LAPLACIAN_2D,
                mode='reflect',
            ) * diffusion_coeff

            # Advection
            advective_flux_x = convolve(
                updated_state,
                np.array([[-1, 0, 1]]),
                mode='reflect',
            ) * advection_coeff[0]
            advective_flux_y = convolve(
                updated_state,
                np.array([[-1], [0], [1]]),
                mode='reflect',
            ) * advection_coeff[1]

            # Update the current state
            updated_state += (laplacian + advective_flux_x + advective_flux_y) * dt

            # # Ensure non-negativity
            # current_states[species] = np.maximum(updated_state, 0)

            # Update time
            t += dt

        return updated_state - state

core.register_process('DiffusionAdvection', DiffusionAdvection)

In [12]:
 n_bins = (4, 4)

initial_glucose = np.random.uniform(low=0, high=20, size=n_bins)
initial_acetate = np.random.uniform(low=0, high=0, size=n_bins)
initial_biomass = np.random.uniform(low=0, high=0.1, size=n_bins)

composite_state = {
    'fields': {
        'glucose': initial_glucose,
        'acetate': initial_acetate,
        'biomass': initial_biomass,
    },
    'diffusion': {
        '_type': 'process',
        'address': 'local:DiffusionAdvection',
        'config': {
            'n_bins': n_bins,
            'bounds': (10, 10),
            'default_diffusion_rate': 1e-1,
            'default_diffusion_dt': 1e-1,
            'diffusion_coeffs': {
                'glucose': 1e-1,
                'acetate': 1e-1,
                'biomass': 1e-1,
            },
            'advection_coeffs': {
                'glucose': (0, 0),
                'acetate': (0, 0),
                'biomass': (0, 0),
            },
        },
        'inputs': {
            'fields': ['fields']
        },
        'outputs': {
            'fields': ['fields']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'fields': 'map',
                'time': 'float',
            }
        },
        'inputs': {
            'fields': ['fields'],
            'time': ['global_time'],
        }
    }
}

sim = Composite({'state': composite_state}, core=core)
# sim.add_emitter()

sim.update({}, 10.0)

data = sim.gather_results()[('emitter',)]

print(data)

[{'fields': {'glucose': array([[ 6.3778075 , 19.22653031, 18.34401593, 13.04759822],
       [10.70063161,  1.34591456,  6.24732449,  8.15221775],
       [15.57146219, 15.31785191,  7.82161606,  9.84856477],
       [13.48377254, 16.31984105, 19.90598232, 10.41022059]]), 'acetate': array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]]), 'biomass': array([[1.22492592e-03, 6.32134456e-02, 8.07464051e-02, 2.18643878e-02],
       [9.20626967e-02, 6.95026218e-05, 1.68679860e-02, 8.04637811e-02],
       [5.71227249e-02, 3.67712875e-02, 6.85828083e-02, 1.84582320e-02],
       [8.11143656e-02, 6.03052318e-02, 8.00668469e-02, 9.89941840e-02]])}, 'time': 0.0}, {'fields': {'glucose': array([[ 7.91099702, 16.42347263, 16.69270908, 13.03018209],
       [10.10414476,  5.32156803,  7.43431668,  8.64163127],
       [14.79321719, 13.72409232,  9.48771404,  9.69451891],
       [13.9305705 , 16.12325668, 17.63446039, 11.17450021]]), 'acetate': array([[0., 0., 0

## COMETS

In [13]:
n_bins = (10, 10)

initial_glucose = np.random.uniform(low=0, high=20, size=n_bins)
initial_acetate = np.random.uniform(low=0, high=0, size=n_bins)
initial_biomass = np.random.uniform(low=0, high=0.1, size=n_bins)

dfba_processes_dict = {}
for i in range(n_bins[0]):
    for j in range(n_bins[1]):
        dfba_processes_dict[f'[{i},{j}]'] = {
            '_type': 'process',
            'address': 'local:DynamicFBA',
            'config': dfba_config(),
            'inputs': {
                'substrates': {
                    'glucose': ['..', 'fields', 'glucose', i, j],
                    'acetate': ['..', 'fields', 'acetate', i, j],
                    'biomass': ['..', 'fields', 'biomass', i, j],
                }
            },
            'outputs': {
                'substrates': {
                    'glucose': ['..', 'fields', 'glucose', i, j],
                    'acetate': ['..', 'fields', 'acetate', i, j],
                    'biomass': ['..', 'fields', 'biomass', i, j]
                }
            }
        }

composite_state = {
    'fields': {
        '_type': 'map',
        '_value': {
            '_type': 'array',
            '_shape': n_bins,
            '_data': 'positive_float'
        },
        'glucose': initial_glucose,
        'acetate': initial_acetate,
        'biomass': initial_biomass,
    },
    'spatial_dfba': dfba_processes_dict,
    'diffusion': {
        '_type': 'process',
        'address': 'local:DiffusionAdvection',
        'config': {
            'n_bins': n_bins,
            'bounds': (10, 10),
            'default_diffusion_rate': 1e-1,
            'default_diffusion_dt': 1e-1,
            'diffusion_coeffs': {
                'glucose': 1e-1,
                'acetate': 1e-1,
                'biomass': 1e-1,
            },
            'advection_coeffs': {
                'glucose': (0, 0),
                'acetate': (0, 0),
                'biomass': (0, 0),
            },
        },
        'inputs': {
            'fields': ['fields']
        },
        'outputs': {
            'fields': ['fields']
        }
    },
    'emitter': {
        '_type': 'step',
        'address': 'local:ram-emitter',
        'config': {
            'emit': {
                'fields': 'map',
                'time': 'float',
            }
        },
        'inputs': {
            'fields': ['fields'],
            'time': ['global_time']
        }
    }
}

sim = Composite({'state': composite_state}, core=core)

sim.update({}, 10.0)

results = sim.gather_results()

print(results)

{('emitter',): [{'fields': {'glucose': array([[ 3.82126173,  3.12636203,  8.22889632, 17.56733378,  9.73343781,
         0.31601036,  8.1821953 ,  3.08994108,  8.06914903, 14.99527261],
       [10.07156141,  3.52828765,  1.31190908,  4.24779386, 18.99041128,
         2.46544413, 14.21736908, 18.56246038, 12.40297421,  2.21408072],
       [18.94420483, 13.70246869,  4.57396732, 10.8122537 ,  7.38989862,
        16.78092721,  4.98489295,  2.59636159, 19.55761987,  0.90474753],
       [11.92760396, 18.17486444, 10.97201684,  2.11927962,  9.38597731,
         6.02713358,  1.53960068, 13.74875238,  5.21003044,  9.44833266],
       [ 9.32633965,  9.76796857, 16.73425389, 19.5525409 ,  5.45817916,
         9.99496607,  5.82465315, 14.28741657, 14.42171748, 10.16918618],
       [ 9.10862658,  6.84781818, 11.62278588, 11.4607051 ,  3.95802964,
         4.72677715, 10.51982268,  2.64383585, 11.64272948, 12.69519414],
       [ 5.69885288,  4.48548957,  6.36648647,  0.20539891, 13.15649957,
      